In [1]:
from parse_EU_act import parse_EU_act

ai_act = parse_EU_act("ai_act/ai-act-draft.pdf")

In [2]:
import re
from auto_label_function import set_to_lower, filter_text, paragraph_to_labeled_sentences, better_studio

set_to_lower()

ai_act = filter_text(ai_act)
ai_act = ai_act.replace("\n", " ")
ai_act = ai_act.replace("  ", " ")
sentences = re.split(r'[.;:]', ai_act)
sentences = [s.strip() for s in sentences if len(s.strip()) > 7]
for i in range(len(sentences)):
    sentences[i] = sentences[i].replace("\n", "")
    while len(sentences[i]) != 0 and (sentences[i][0] == " "):
        sentences[i] = sentences[i][1:]
sentences[:5]

['INTRODUCTION The Commission adopted the proposal for a Regulation laying down harmonised rules on artificial intelligence (Artificial Intelligence Act, hereinafter',
 'the AI Act) on 21 April 2021',
 'The Council unanimously adopted its General Approach on the proposal on 6 December 2022, while the European Parliament (hereinafter',
 'the EP) confirmed its position in a plenary vote on 14 June 2023',
 'On 14 June 2023, 18 July 2023, 2-3 October 2023 and 24 October 2023 the first four political trilogues were held, during which some of the less controversial parts of the proposal were agreed and compromise was also found on the provisions concerning measures in support of innovation, as well as and on the mechanism for classification of AI systems as high-risk']

In [3]:
auto_ner_output = [better_studio(*paragraph_to_labeled_sentences(sentence)) for sentence in sentences]
auto_ner_output[0]

[{'id': 2,
  'data': {'text': 'INTRODUCTION The Commission adopted the proposal for a Regulation laying down harmonised rules on artificial intelligence (Artificial Intelligence Act, hereinafter'},
  'predictions': [{'result': [{'value': {'start': 13,
       'end': 27,
       'text': 'The Commission',
       'labels': ['ORG']},
      'id': 'kzgwhaafdmlmctfa',
      'from_name': 'label',
      'to_name': 'text',
      'type': 'labels',
      'origin': 'auto'},
     {'value': {'start': 55,
       'end': 65,
       'text': 'Regulation',
       'labels': ['DOC']},
      'id': 'ksxsjkxvcullhhwz',
      'from_name': 'label',
      'to_name': 'text',
      'type': 'labels',
      'origin': 'auto'},
     {'value': {'start': 98,
       'end': 121,
       'text': 'artificial intelligence',
       'labels': ['ALG']},
      'id': 'ohfhgntderbvcfhl',
      'from_name': 'label',
      'to_name': 'text',
      'type': 'labels',
      'origin': 'auto'},
     {'value': {'start': 123,
       'end': 146,

In [4]:
# Filter the dataset for empty entities
for sentence in auto_ner_output:
    indices_to_remove = []
    for i, entity in enumerate(sentence[0]["predictions"][0]["result"]):
        if not entity["value"]["text"].strip():
            indices_to_remove.append(i)
    for index in sorted(indices_to_remove, reverse=True):
        del sentence[0]["predictions"][0]["result"][index]

In [5]:
auto_ner_output[66]

[{'id': 2,
  'data': {'text': 'An advisory forum will provide stakeholder input to the Commission (including the AI Office) and to the AI Board'},
  'predictions': [{'result': [{'value': {'start': 3,
       'end': 17,
       'text': 'advisory forum',
       'labels': ['ORG']},
      'id': 'hszstbhouiuxfwcz',
      'from_name': 'label',
      'to_name': 'text',
      'type': 'labels',
      'origin': 'auto'},
     {'value': {'start': 52,
       'end': 66,
       'text': 'the Commission',
       'labels': ['ORG']},
      'id': 'qxldwnzsamjddjnf',
      'from_name': 'label',
      'to_name': 'text',
      'type': 'labels',
      'origin': 'auto'},
     {'value': {'start': 78,
       'end': 91,
       'text': 'the AI Office',
       'labels': ['ORG']},
      'id': 'hqsxpdxzidldsmix',
      'from_name': 'label',
      'to_name': 'text',
      'type': 'labels',
      'origin': 'auto'},
     {'value': {'start': 104, 'end': 106, 'text': 'AI', 'labels': ['ALG']},
      'id': 'feoioopfqreusnne',

In [6]:
from auto_ner_to_docred import auto_ner_to_docred

# for i in range(len(auto_ner_output)):
#     print(i)
#     auto_ner_to_docred(auto_ner_output[i])

docred_format = [auto_ner_to_docred(sentence)[0] for sentence in auto_ner_output]
docred_format[4]

{'vertexSet': [[{'pos': [62, 64],
    'type': 'SYS',
    'sent_id': 0,
    'name': 'AI systems'}],
  [{'pos': [65, 66], 'type': 'ETH', 'sent_id': 0, 'name': 'high-risk'}]],
 'sents': [['On',
   '14',
   'June',
   '2023',
   ',',
   '18',
   'July',
   '2023',
   ',',
   '2-3',
   'October',
   '2023',
   'and',
   '24',
   'October',
   '2023',
   'the',
   'first',
   'four',
   'political',
   'trilogues',
   'were',
   'held',
   ',',
   'during',
   'which',
   'some',
   'of',
   'the',
   'less',
   'controversial',
   'parts',
   'of',
   'the',
   'proposal',
   'were',
   'agreed',
   'and',
   'compromise',
   'was',
   'also',
   'found',
   'on',
   'the',
   'provisions',
   'concerning',
   'measures',
   'in',
   'support',
   'of',
   'innovation',
   ',',
   'as',
   'well',
   'as',
   'and',
   'on',
   'the',
   'mechanism',
   'for',
   'classification',
   'of',
   'AI',
   'systems',
   'as',
   'high-risk']],
 'labels': [{'r': '', 'h': '', 't': '', 'evidence': 

In [7]:
docred_format_filtered = [element for element in docred_format if len(element["vertexSet"]) > 1 and len(element["vertexSet"][0]) > 1]

In [8]:
docred_format_filtered[4]

{'vertexSet': [[{'pos': [2, 4],
    'type': 'ORG',
    'sent_id': 0,
    'name': 'law enforcement'},
   {'pos': [13, 15], 'type': 'ORG', 'sent_id': 0, 'name': 'law enforcement'}],
  [{'pos': [28, 30],
    'type': 'SYS',
    'sent_id': 0,
    'name': ' conformity assessmen'}],
  [{'pos': [36, 37], 'type': 'ACT', 'sent_id': 0, 'name': ' testin'}],
  [{'pos': [39, 42],
    'type': 'SYS',
    'sent_id': 0,
    'name': ' high-risk AI system'}]],
 'sents': [['Exceptions',
   'for',
   'law',
   'enforcement',
   'authorities',
   'The',
   'compromise',
   'agreement',
   'preserves',
   'all',
   'the',
   'exceptions',
   'for',
   'law',
   'enforcement',
   'authorities',
   'that',
   'were',
   'added',
   'in',
   'the',
   'Council’s',
   'revised',
   'mandate',
   ',',
   'including',
   'derogation',
   'from',
   'conformity',
   'assessment',
   'and',
   'the',
   'possibility',
   'to',
   'start',
   'real',
   'world',
   'testing',
   'of',
   'high-risk',
   'AI',
   'syst

In [9]:
for element in docred_format_filtered:
    element.pop("labels", None)
docred_format_filtered[0]

{'vertexSet': [[{'pos': [10, 11],
    'type': 'ETH',
    'sent_id': 0,
    'name': 'risks'},
   {'pos': [17, 18], 'type': 'ETH', 'sent_id': 0, 'name': 'risks'}],
  [{'pos': [14, 15], 'type': 'DOC', 'sent_id': 0, 'name': 'Regulation'}],
  [{'pos': [19, 20], 'type': 'ETH', 'sent_id': 0, 'name': 'health'}],
  [{'pos': [21, 22], 'type': 'ETH', 'sent_id': 0, 'name': 'safety'}],
  [{'pos': [24, 25], 'type': 'ETH', 'sent_id': 0, 'name': 'rights'}]],
 'sents': [['However',
   ',',
   'all',
   'subsequent',
   'references',
   'in',
   'the',
   'text',
   'to',
   'the',
   'risks',
   'addressed',
   'by',
   'the',
   'Regulation',
   'only',
   'include',
   'risks',
   'to',
   'health',
   ',',
   'safety',
   'and',
   'fundamental',
   'rights',
   ',',
   'in',
   'line',
   'with',
   'the',
   'Council’s',
   'mandate']],
 'title': 'Legal text about AI'}

In [10]:
import json

with open("ai_act_docred_format.json", "w") as outfile:
    json.dump(docred_format_filtered, outfile)

In [11]:
with open("short_ai_act_docred_format.json", "w") as outfile:
    json.dump(docred_format_filtered[:10], outfile, indent=4)